In [30]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests as rq
from github import Github, GithubException
import time
from shutil import copyfile
from pymongo import MongoClient
import re
from bson.objectid import ObjectId
import pprint

In [2]:
client = MongoClient()
db = client['repos-database']
dockers = db['docker_repos']

In [40]:
cursor = dockers.find()
# Expand the cursor and construct the DataFrame
df =  pd.DataFrame(list(cursor))
print(df.head(1))
'''
for post in dockers.find():
    pprint.pprint(post)
    break
'''

         Data Criação         Data Update  \
0 2016-11-25 20:25:47 2016-11-25 20:25:50   

                                          Dockerfile  \
0  {'FROM': ['microsoft/dotnet:latest'], 'MAINTAI...   

                                          Linguagens  \
0  [TypeScript 32.2%, C# 25.2%, HTML 18.6%, JavaS...   

                                      Repo link                Repositório  \
0  https://github.com/goomoopunch/BiggityBudget  goomoopunch/BiggityBudget   

                        _id  
0  5a2b73938b062b10d17ac2e4  


'\nfor post in dockers.find():\n    pprint.pprint(post)\n    break\n'